# Data Processing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
nlp = spacy.load('en_core_web_md')

In [3]:
data_yelp = pd.read_csv('yelp_labelled.txt', sep='\t',header=None)
columan_name = ['Review', 'Sentiment']
data_yelp.columns = columan_name
#reassign column names

In [4]:
#inspect data 
#sentiment - 1 for positive reviews 
# - 0 for negative reviews 

In [5]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [11]:
data_amazon = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.columns = columan_name
data_imdb = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)
data_imdb.columns = columan_name

In [12]:
# Append all the data in a single dataframe
data = data_yelp.append([data_amazon, data_imdb],ignore_index=True)

In [13]:
data.shape

(2748, 2)

In [14]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [15]:
#check distribution of the data - approximately the same 
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [16]:
# check for null values
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [22]:
#assign x and y variables 
x = data['Review']
y = data['Sentiment']

## Text Processing
create a tokenizer for tfidf

In [23]:
import string
punct = string.punctuation
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS) # list of stopwords

In [38]:
def text_data_cleaning(sentence):
    doc = nlp(sentence)

    tokens = [] # list of tokens
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
 
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

# if root form of that word is not pronoun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, 
#because there is no lemma for proper noun

In [39]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

In [40]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [41]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
# tokenizer=text_data_cleaning, tokenization will be done according to this function

In [42]:
classifier = LinearSVC()

# Model

In [43]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [44]:
x_train.shape, x_test.shape

((2198,), (550,))

In [45]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])
# it will first do vectorization and then it will do classification

In [46]:
clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7fed05ff5160>)),
                ('clf', LinearSVC())])

In [47]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [48]:
y_pred = clf.predict(x_test)

In [49]:
# confusion_matrix
confusion_matrix(y_test, y_pred)

array([[199,  80],
       [ 52, 219]])

In [50]:
# classification_report
print(classification_report(y_test, y_pred))
# we are getting almost 77% accuracy

              precision    recall  f1-score   support

           0       0.79      0.71      0.75       279
           1       0.73      0.81      0.77       271

    accuracy                           0.76       550
   macro avg       0.76      0.76      0.76       550
weighted avg       0.76      0.76      0.76       550



In [51]:
accuracy_score(y_test, y_pred)

0.76

In [52]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])
#positive review as expected

array([1])